In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# Create a path variable (Adjust this to your specific project path if needed)
path = r'C:\Users\howel\OneDrive\Instacart Basket Analysis'

# Import the ords_prods_merge dataframe
orders_products_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [2]:
# Group by 'department_id' and calculate the mean of 'order_number'
orders_products_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


The results from the subset were slightly different because they only included the first 1 million rows. This limited dataset was not fully representative of the entire population. The results from the entire dataframe (shown above) are more accurate because they include every single order.

In [3]:
# Create the max_order column using the string 'max' to avoid the warning
orders_products_merged['max_order'] = orders_products_merged.groupby(['user_id'])['order_number'].transform('max')

In [4]:
# Assign 'Loyal customer' label
orders_products_merged.loc[orders_products_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

# Assign 'Regular customer' label
orders_products_merged.loc[(orders_products_merged['max_order'] <= 40) & (orders_products_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

# Assign 'New customer' label
orders_products_merged.loc[orders_products_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [5]:
# Check the frequency of the new flag
orders_products_merged['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [6]:
# Group by 'loyalty_flag' and calculate statistics for 'prices'
orders_products_merged.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

In [7]:
# Create the average_price column using the string 'mean'
orders_products_merged['average_price'] = orders_products_merged.groupby(['user_id'])['prices'].transform('mean')

In [8]:
# Assign 'Low spender' label
orders_products_merged.loc[orders_products_merged['average_price'] < 10, 'spending_flag'] = 'Low spender'

# Assign 'High spender' label
orders_products_merged.loc[orders_products_merged['average_price'] >= 10, 'spending_flag'] = 'High spender'

In [9]:
orders_products_merged['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [10]:
# Create the median_days column using the string 'median'
orders_products_merged['median_days'] = orders_products_merged.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [11]:
# Assign 'Non-frequent customer' label
orders_products_merged.loc[orders_products_merged['median_days'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'

# Assign 'Regular customer' label
orders_products_merged.loc[(orders_products_merged['median_days'] > 10) & (orders_products_merged['median_days'] <= 20), 'order_frequency_flag'] = 'Regular customer'

# Assign 'Frequent customer' label
orders_products_merged.loc[orders_products_merged['median_days'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [12]:
orders_products_merged['order_frequency_flag'].value_counts(dropna = False)

order_frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

In [13]:
# Export the dataframe to pickle
orders_products_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))